In [3]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats
from statsmodels.stats import weightstats as ws
from pydataset import data

In [18]:
df = data('iris')

# T-test

Assumptions: Independent samples are taken from a normal distribution. In this case, the normalized version of $\bar{X}$ using the sample standard deviation $S$ instead of the population standard deviation $\sigma$ follows the $t$-distribution with $n-1$ Degree of Freedom (dof).
- It is robust to the normality assumption

Test statistic: $T=\frac{\bar{X}-\mu_0}{S/\sqrt{n}}$



## 1-sample *t*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [3]:
pg.ttest(
    x=df['Sepal.Length'], 
    y=5, 
    paired=False,
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,12.473257,149,two-sided,6.670742e-25,"[5.71, 5.98]",1.018437,5.88e+21,1.0


## 2-sample *t*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [4]:
pg.ttest(
    x=df.loc[df['Species']=='setosa','Sepal.Length'], 
    y=df.loc[df['Species']=='versicolor','Sepal.Length'],
    paired=False, # True for paired samples
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-10.520986,98,two-sided,8.985235e-18,"[-1.11, -0.75]",2.104197,4.19e+14,1.0


# *Z*-Test

Assumptions: When sample size is large, the distribution of sample mean $(\bar{X})$ is approximately normal with
- $E(\bar{X}) = \mu$ 
- $Var(\bar{X}) = \sigma^2/n$

Test statistic: $Z=\frac{\bar{X}-\mu_0}{\sigma/\sqrt{n}}$

## 1-sample *Z*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [5]:
zstat, pval = ws.ztest(
    x1=df['Sepal.Length'], 
    x2=None, 
    value=5.8,
    alternative='two-sided' #'larger', 'smaller'
)
print("z-stat:", zstat, "p-value:", pval)

z-stat: 0.6409183514112012 p-value: 0.5215757321117633


## 2-sample *Z*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [7]:
zstat, pval = ws.ztest(
    x1 = df.loc[df['Species']=='setosa','Sepal.Length'], 
    x2 = df.loc[df['Species']=='versicolor','Sepal.Length'],
    value = 0,
    alternative = 'two-sided' #'larger', 'smaller'
)
print("z-stat:", zstat, "p-value:", pval)

z-stat: -10.52098626754911 p-value: 6.914595261207391e-26


# Chi-Square test

## Hypotheses Concerning a Population Variance

- Assumption: Independent samples are taken from a normal distribution. In this case, $\chi^2 = \frac{(n-1)S^2}{\sigma^2}$
    - not robust to the normality assumption
- Test Statistic: $\chi^2 = \frac{(n-1)S^2}{\sigma_0^2}$
- Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\sigma=0.6857$
- $H_a$: $\sigma\neq0.6857$

In [9]:
def calculate_chi2_stat(x, var0):
    return (len(x)-1)*np.var(x)/var0

def var_test(x, var0, tail='two-sided'):
    chi2_stat = calculate_chi2_stat(x, var0)
    left_p = stats.chi2.cdf(chi2_stat, df=len(x)-1) # P(X<=x)
    if tail=='greater':
        return chi2_stat, 1-left_p
    if tail=='less':
        return chi2_stat, left_p
    else:
        return chi2_stat, (left_p if left_p<0.5 else 1-left_p) * 2

In [10]:
chi2_stat, pval = var_test(
    x = df['Sepal.Length'], 
    var0 = 1,
    tail='two-sided'
)
print("chi2-stat:", chi2_stat, "p-value:", pval)

chi2-stat: 101.48721111111111 p-value: 0.002101176207511642


## Comparison of Two Variances

- Assumption: Independent samples are taken from a normal distribution. In this case: $F = \frac{(n-1)S_1^2/\sigma_1^2}{(n-1)S_2^2/\sigma_2^2} = \frac{S_1^2/\sigma_1^2}{S_2^2/\sigma_2^2}$
    - not robust to the normality assumption
- Test Statistic: $F = \frac{S_1^2}{S_2^2}$

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $\sigma_1=\sigma_2$
- $H_a$: $\sigma_1\neq\sigma_2$

In [11]:
def var_F_test(x, y, tail='two-sided'):
    f_stat = np.var(x)/np.var(y)
    left_p = stats.f.cdf(f_stat, dfn=len(x)-1, dfd=len(y)-1) # P(X<=x)
    if tail=='greater':
        return f_stat, 1-left_p
    if tail=='less':
        return f_stat, left_p
    else:
        return f_stat, (left_p if left_p<0.5 else 1-left_p) * 2

In [12]:
f_stat, pval = var_F_test(
    x = df.loc[df['Species']=='setosa','Sepal.Length'], 
    y = df.loc[df['Species']=='versicolor','Sepal.Length'],
    tail = 'two-tail'
)
print("F-stat:", f_stat, "p-value:", pval)

F-stat: 0.4663429131686986 p-value: 0.008657188362699816


## Goodness-of-Fit Test (Categorical Data)

- Rule of thumb: All expected cell counts are at least five.
- For an $r*c$ contingency table, the Test Statistic, $\chi^2 = \sum_{j=1}^{r} \sum_{i=1}^{c} \frac{[n_{ij}-E(n_{ij})]^2}{E(n_{ij})}$
    - where $E(n_{ij}) = \frac{row\_total*column\_total}{n}$
- dof: The number of cells minus 1 df for each independent linear restriction or each parameter estimated using MLE.
    - For an $r*c$ contingency table, $dof = (r-1)(c-1)$
- Variables:
    - Categorical variable: Species
    - Target variable: sepal_width_atleast3.1
- $H_0$: The categorical variables are not correlated.
- $H_a$: The categorical variables are correlated.

In [9]:
df['sepal_width_atleast3_1'] = df['Sepal.Width']>=3.1
df.groupby('Species')['sepal_width_atleast3.1'].value_counts()

Species     sepal_width_atleast3.1
setosa      True                      42
            False                      8
versicolor  False                     42
            True                       8
virginica   False                     33
            True                      17
Name: sepal_width_atleast3.1, dtype: int64

In [123]:
expected, observed, stats = pg.chi2_independence(df, x='Species', y='sepal_width_atleast3.1')
display(expected.style.set_caption("Expected"), observed.style.set_caption("observed"), stats)

sepal_width_atleast3.1,False,True
Species,,
setosa,27.666667,22.333333
versicolor,27.666667,22.333333
virginica,27.666667,22.333333


sepal_width_atleast3.1,False,True
Species,,
setosa,8,42
versicolor,42,8
virginica,33,17


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,50.224780,2.0,1.241160e-11,0.578647,0.999999
1,cressie-read,0.666667,50.918092,2.0,8.775601e-12,0.582627,0.999999
2,log-likelihood,0.000000,54.196713,2.0,1.703466e-12,0.601092,1.000000
3,freeman-tukey,-0.500000,58.722124,2.0,1.772771e-13,0.625684,1.000000
4,mod-log-likelihood,-1.000000,65.637935,2.0,5.583462e-15,0.661503,1.000000
5,neyman,-2.000000,90.663462,2.0,2.054373e-20,0.777447,1.000000


# ANOVA

## One-way ANOVA
- Assumption: The $k$ populations are normally distributed with a same variance.
    - The test is still robust with moderate departures from the normality and equal variances assumption.
    - The equal variances assumption is less critical if sample size of all populations are equal.
- Varations:
    - welch_anova: More reliable with unequal variances and/or unequal sample sizes.
    - ancova: Statistically controlling for the effects of other continuous variables that are not of primary interest.
    - rm_anova: Repeated measures on a same subject (analogous to paired samples).
- Test Statistic: $F = MST/MSE$, large F indicates that $H_0$ should be rejected.
- Variables:
    - Categorical variable 1: Species
    - Categorical variable 2: sepal_width_atleast3.1
    - Target variable: Sepal.Length
- $H_0$: $\mu_1 = \mu_2 = ... = \mu_n$
- $H_a$: At least 1 of the population mean is significantly different from another.

In [5]:
pg.anova(
    data=df, 
    dv='Sepal.Length', 
    between='Species', 
    detailed=True
)

,Source,SS,DF,MS,F,p-unc,np2
0,Species,63.212133,2,31.606067,119.264502,1.669669e-31,0.618706
1,Within,38.956200,147,0.265008,NaN,NaN,NaN


## Two-way ANOVA

Similar to one-way ANOVA but it handles two categorical variables and their interactions, each variable and the interaction have a *p*-value to differentiate if the variable/interaction lead to different means.

In [14]:
pg.anova(
    data=df.rename(columns={'Sepal.Length':'sepal_len'}), 
    dv='sepal_len', 
    between=['Species','sepal_width_atleast3_1'], 
    detailed=True
)

,Source,SS,DF,MS,F,p-unc,np2
0,Species,65.086000,2.0,32.543000,139.705044,1.884920e-34,0.659904
1,sepal_width_atleast3_1,4.963994,1.0,4.963994,21.310114,8.574314e-06,0.128910
2,Species * sepal_width_atleast3_1,0.448736,2.0,0.224368,0.963197,3.841159e-01,0.013201
3,Residual,33.543470,144.0,0.232941,NaN,NaN,NaN


## Post-Hoc Tests

- If classic *t*-tests are used to perform post-ANOVA pairwise comparisons, the probability of Type-I error ($\alpha$) of the whole experiment increases with respect to the number of pairwise comparisons made. 
    - $\alpha_c = 1-(1-\alpha)^c$, where $c$ denotes the number of pairwise comparisons.
- This is the reason of applying post-hoc tests, which is able to maintain the experiment-wise $\alpha$ by lowering the statistical power (adjusting the *p*-values and the confidence intervals), i.e. increasing the probability of Type-II error.
- Interpretation of adjusted *p*-values: There is a probability, *p* to observe such sample if the $H_0$ of the ANOVA is true.

### Tukey's Honestly Significant Difference (HSD) Test
- Best for balanced one-way ANOVA.

In [20]:
pg.pairwise_tukey(
    data=df, 
    dv='Sepal.Length', 
    between='Species'
)

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,setosa,versicolor,5.006,5.936,-0.930,0.102958,-9.032819,0.001,-1.792703
1,setosa,virginica,5.006,6.588,-1.582,0.102958,-15.365506,0.001,-3.049522
2,versicolor,virginica,5.936,6.588,-0.652,0.102958,-6.332686,0.001,-1.256820


### Games-Howell Test
- Best for Welch ANOVA.
- Compared to the Tukey-HSD test, the Games-Howell test uses different pooled variances for each pair of variables instead of the same pooled variance.

In [21]:
pg.pairwise_gameshowell(
    data=df, 
    dv='Sepal.Length', 
    between='Species'
)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,setosa,versicolor,5.006,5.936,-0.930,0.088395,-10.520986,86.538002,0.001,-2.088053
1,setosa,virginica,5.006,6.588,-1.582,0.102819,-15.386196,76.515867,0.001,-3.053629
2,versicolor,virginica,5.936,6.588,-0.652,0.115825,-5.629165,94.025486,0.001,-1.117195


### Pairwise T-test
- Compute adjusted *p*-values in multiple comparisons.
- Flexible for repeated measures ANOVA.

In [22]:
df_mixed_anova = pg.read_dataset('mixed_anova.csv')

In [28]:
# Independent samples
pg.pairwise_ttests(
    data=df_mixed_anova,
    dv='Scores', 
    between=['Time'],    
    padjust='holm',
    parametric=True
)

,Contrast,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,Time,August,January,False,True,-1.805747,118.0,two-sided,0.073507,0.147015,holm,0.839,-0.327583
1,Time,August,June,False,True,-2.659961,118.0,two-sided,0.008902,0.026705,holm,4.499,-0.482547
2,Time,January,June,False,True,-0.934454,118.0,two-sided,0.351978,0.351978,holm,0.288,-0.169520


In [29]:
# Paired samples
pg.pairwise_ttests(
    data=df_mixed_anova, 
    dv='Scores', 
    within='Time', 
    subject='Subject', 
    padjust='holm',
    parametric=True               
)

,Contrast,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,Time,August,January,True,True,-1.740370,59.0,two-sided,0.087008,0.174015,holm,0.582,-0.327583
1,Time,August,June,True,True,-2.743238,59.0,two-sided,0.008045,0.024134,holm,4.232,-0.482547
2,Time,January,June,True,True,-1.023620,59.0,two-sided,0.310194,0.310194,holm,0.232,-0.169520


In [30]:
# Paired + Unpaired
pg.pairwise_ttests(
    data=df_mixed_anova, 
    dv='Scores', 
    within='Time',
    subject='Subject', 
    between='Group',
    padjust='holm', 
)

,Contrast,Time,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,Time,-,August,January,True,True,-1.740370,59.0,two-sided,0.087008,0.174015,holm,0.582,-0.327583
1,Time,-,August,June,True,True,-2.743238,59.0,two-sided,0.008045,0.024134,holm,4.232,-0.482547
2,Time,-,January,June,True,True,-1.023620,59.0,two-sided,0.310194,0.310194,holm,0.232,-0.169520
3,Group,-,Control,Meditation,False,True,-2.247601,58.0,two-sided,0.028420,NaN,NaN,2.096,-0.572791
4,Time * Group,August,Control,Meditation,False,True,0.316022,58.0,two-sided,0.753120,0.753120,holm,0.274,0.080537
5,Time * Group,January,Control,Meditation,False,True,-1.433725,58.0,two-sided,0.157020,0.314040,holm,0.619,-0.365379
6,Time * Group,June,Control,Meditation,False,True,-2.744291,58.0,two-sided,0.008058,0.024173,holm,5.593,-0.699371


In [32]:
# Interaction among between
pg.pairwise_ttests(
    data=df_mixed_anova, 
    dv='Scores', 
    between=['Group', 'Time'],
    padjust='holm', 
)

,Contrast,Group,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,Group,-,Control,Meditation,False,True,-2.289903,178.0,two-sided,0.023200,NaN,NaN,1.813,-0.339918
1,Time,-,August,January,False,True,-1.805747,118.0,two-sided,0.073507,0.147015,holm,0.839,-0.327583
2,Time,-,August,June,False,True,-2.659961,118.0,two-sided,0.008902,0.026705,holm,4.499,-0.482547
3,Time,-,January,June,False,True,-0.934454,118.0,two-sided,0.351978,0.351978,holm,0.288,-0.169520
4,Group * Time,Control,August,January,False,True,-0.382627,58.0,two-sided,0.703395,1.000000,holm,0.279,-0.097511
5,Group * Time,Control,August,June,False,True,-0.291809,58.0,two-sided,0.771474,1.000000,holm,0.272,-0.074366
6,Group * Time,Control,January,June,False,True,0.044732,58.0,two-sided,0.964475,1.000000,holm,0.263,0.011400
7,Group * Time,Meditation,August,January,False,True,-2.187830,58.0,two-sided,0.032725,0.163625,holm,1.884,-0.557559
8,Group * Time,Meditation,August,June,False,True,-4.040093,58.0,two-sided,0.000159,0.000955,holm,148.302,-1.029600
9,Group * Time,Meditation,January,June,False,True,-1.441731,58.0,two-sided,0.154756,0.619025,holm,0.625,-0.367419


In [33]:
# Ignore interaction
pg.pairwise_ttests(
    data=df_mixed_anova, 
    dv='Scores', 
    between=['Group', 'Time'],
    padjust='holm', 
    interaction=False
)

,Contrast,A,B,Paired,Parametric,T,dof,Tail,p-unc,p-corr,p-adjust,BF10,hedges
0,Group,Control,Meditation,False,True,-2.289903,178.0,two-sided,0.023200,NaN,NaN,1.813,-0.339918
1,Time,August,January,False,True,-1.805747,118.0,two-sided,0.073507,0.147015,holm,0.839,-0.327583
2,Time,August,June,False,True,-2.659961,118.0,two-sided,0.008902,0.026705,holm,4.499,-0.482547
3,Time,January,June,False,True,-0.934454,118.0,two-sided,0.351978,0.351978,holm,0.288,-0.169520
